In [ ]:
# Importar librerías
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from shapely.geometry import Point, box
from scipy import stats
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Librerías cargadas")

In [ ]:
# Cargar librerías espaciales avanzadas
try:
    import libpysal
    from libpysal.weights import Queen, KNN, DistanceBand
    from esda.getisord import G_Local
    from pointpats import PointPattern, PoissonPointProcess, as_window
    from pointpats.ripley import k_function, g_function, f_function
    ADVANCED_AVAILABLE = True
    print("✅ Librerías avanzadas disponibles")
except ImportError as e:
    ADVANCED_AVAILABLE = False
    print(f"⚠️ Algunas librerías no disponibles: {e}")
    print("Instalar con: pip install libpysal esda pointpats")

## 1. Cargar Datos

In [ ]:
import os

DATA_PATH = '../data/raw/isla_de_pascua'

# Cargar datos
boundary = gpd.read_file(os.path.join(DATA_PATH, 'isla_de_pascua_boundary.geojson'))
buildings = gpd.read_file(os.path.join(DATA_PATH, 'isla_de_pascua_buildings.geojson'))
amenities = gpd.read_file(os.path.join(DATA_PATH, 'isla_de_pascua_amenities.geojson'))
streets = gpd.read_file(os.path.join(DATA_PATH, 'isla_de_pascua_streets.geojson'))

# Proyectar a UTM para cálculos métricos
CRS_UTM = 'EPSG:32719'  # UTM 19S
boundary_utm = boundary.to_crs(CRS_UTM)
buildings_utm = buildings.to_crs(CRS_UTM)
amenities_utm = amenities.to_crs(CRS_UTM)

# Calcular áreas de edificios
buildings_utm['area_m2'] = buildings_utm.geometry.area

print(f"✅ Datos cargados:")
print(f"   - Edificios: {len(buildings)}")
print(f"   - Amenidades: {len(amenities)}")
print(f"   - Calles: {len(streets)}")

## 2. Análisis de Patrones Puntuales

In [ ]:
# Obtener centroides de edificios
building_centroids = buildings_utm.copy()
building_centroids['geometry'] = building_centroids.geometry.centroid

# Extraer coordenadas
coords = np.array([[p.x, p.y] for p in building_centroids.geometry])

print(f"📍 Centroides extraídos: {len(coords)}")
print(f"   Rango X: {coords[:,0].min():.0f} - {coords[:,0].max():.0f} m")
print(f"   Rango Y: {coords[:,1].min():.0f} - {coords[:,1].max():.0f} m")

In [ ]:
# Análisis de Nearest Neighbor
from scipy.spatial import distance_matrix

# Calcular distancias
dist_matrix = distance_matrix(coords, coords)
np.fill_diagonal(dist_matrix, np.inf)  # Ignorar distancia a sí mismo

# Distancia al vecino más cercano
nn_distances = dist_matrix.min(axis=1)

# Estadísticas
mean_nn = nn_distances.mean()
std_nn = nn_distances.std()

# Índice de Nearest Neighbor
area = boundary_utm.geometry.area.values[0]
n = len(coords)
expected_nn = 0.5 * np.sqrt(area / n)  # Distribución aleatoria
nn_index = mean_nn / expected_nn

print("📊 ANÁLISIS DE VECINO MÁS CERCANO")
print("=" * 50)
print(f"Distancia promedio al vecino más cercano: {mean_nn:.2f} m")
print(f"Distancia esperada (aleatorio): {expected_nn:.2f} m")
print(f"Índice de Nearest Neighbor: {nn_index:.4f}")
print("\n📌 Interpretación:")
if nn_index < 1:
    print(f"   → Patrón AGRUPADO (clustered) - NNI = {nn_index:.2f} < 1")
elif nn_index > 1:
    print(f"   → Patrón DISPERSO (dispersed) - NNI = {nn_index:.2f} > 1")
else:
    print(f"   → Patrón ALEATORIO (random) - NNI ≈ 1")

In [ ]:
# Histograma de distancias al vecino más cercano
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histograma
ax1 = axes[0]
ax1.hist(nn_distances, bins=50, color='steelblue', edgecolor='white', alpha=0.7)
ax1.axvline(mean_nn, color='red', linestyle='--', linewidth=2, label=f'Media: {mean_nn:.1f} m')
ax1.axvline(expected_nn, color='green', linestyle='--', linewidth=2, label=f'Esperado: {expected_nn:.1f} m')
ax1.set_xlabel('Distancia al vecino más cercano (m)')
ax1.set_ylabel('Frecuencia')
ax1.set_title('Distribución de Distancias NN')
ax1.legend()

# Mapa de distancias
ax2 = axes[1]
boundary_utm.plot(ax=ax2, facecolor='lightgray', edgecolor='black')
scatter = ax2.scatter(coords[:,0], coords[:,1], c=nn_distances, 
                      cmap='RdYlBu_r', s=5, alpha=0.7)
plt.colorbar(scatter, ax=ax2, label='Distancia NN (m)')
ax2.set_title('Distancia al Vecino Más Cercano')
ax2.set_axis_off()

plt.tight_layout()
plt.savefig('../outputs/08_nearest_neighbor.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Getis-Ord Gi* (Hot Spot Analysis)

In [ ]:
# Crear grilla para hot spot analysis
minx, miny, maxx, maxy = boundary_utm.total_bounds
cell_size = 200  # metros

grid_cells = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))
        y += cell_size
    x += cell_size

grid = gpd.GeoDataFrame(geometry=grid_cells, crs=CRS_UTM)
grid = grid[grid.intersects(boundary_utm.unary_union)]
grid = grid.reset_index(drop=True)

# Contar edificios por celda
grid['building_count'] = 0
grid['total_area'] = 0.0

for idx, cell in grid.iterrows():
    buildings_in_cell = buildings_utm[buildings_utm.geometry.centroid.within(cell.geometry)]
    grid.loc[idx, 'building_count'] = len(buildings_in_cell)
    grid.loc[idx, 'total_area'] = buildings_in_cell['area_m2'].sum()

print(f"✅ Grilla creada: {len(grid)} celdas de {cell_size}m x {cell_size}m")

In [ ]:
# Calcular Getis-Ord Gi*
if ADVANCED_AVAILABLE:
    # Filtrar celdas con datos
    analysis_grid = grid[grid['building_count'] > 0].copy().reset_index(drop=True)
    
    if len(analysis_grid) > 10:
        # Crear matriz de pesos por distancia
        w = DistanceBand.from_dataframe(analysis_grid, threshold=500)  # 500m
        w.transform = 'r'
        
        # Calcular Gi*
        y = analysis_grid['building_count'].values
        gi_star = G_Local(y, w, star=True)
        
        analysis_grid['gi_z'] = gi_star.Zs
        analysis_grid['gi_p'] = gi_star.p_sim
        
        # Clasificar hot/cold spots
        def classify_gi(row):
            z = row['gi_z']
            p = row['gi_p']
            if p > 0.05:
                return 'No Significativo'
            elif z > 2.58:
                return 'Hot Spot (99%)'
            elif z > 1.96:
                return 'Hot Spot (95%)'
            elif z > 1.65:
                return 'Hot Spot (90%)'
            elif z < -2.58:
                return 'Cold Spot (99%)'
            elif z < -1.96:
                return 'Cold Spot (95%)'
            elif z < -1.65:
                return 'Cold Spot (90%)'
            else:
                return 'No Significativo'
        
        analysis_grid['spot_type'] = analysis_grid.apply(classify_gi, axis=1)
        
        print("📊 ANÁLISIS GETIS-ORD Gi*")
        print("=" * 50)
        print(analysis_grid['spot_type'].value_counts())
    else:
        print("⚠️ No hay suficientes celdas para análisis")
else:
    print("⚠️ Realizando análisis alternativo sin libpysal")
    # Análisis alternativo usando z-scores simples
    analysis_grid = grid[grid['building_count'] > 0].copy().reset_index(drop=True)
    mean_count = analysis_grid['building_count'].mean()
    std_count = analysis_grid['building_count'].std()
    analysis_grid['z_score'] = (analysis_grid['building_count'] - mean_count) / std_count
    
    def classify_z(z):
        if z > 1.96:
            return 'Alta Densidad'
        elif z < -1.96:
            return 'Baja Densidad'
        else:
            return 'Densidad Normal'
    
    analysis_grid['spot_type'] = analysis_grid['z_score'].apply(classify_z)
    print(analysis_grid['spot_type'].value_counts())

In [ ]:
# Mapa de Hot Spots
fig, ax = plt.subplots(figsize=(14, 10))

# Colores
colors = {
    'Hot Spot (99%)': '#d7191c',
    'Hot Spot (95%)': '#fdae61',
    'Hot Spot (90%)': '#fee08b',
    'No Significativo': '#eeeeee',
    'Cold Spot (90%)': '#d1e5f0',
    'Cold Spot (95%)': '#67a9cf',
    'Cold Spot (99%)': '#2166ac',
    'Alta Densidad': '#d7191c',
    'Densidad Normal': '#eeeeee',
    'Baja Densidad': '#2166ac'
}

# Dibujar límite
boundary_utm.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

# Dibujar celdas
for spot_type in analysis_grid['spot_type'].unique():
    subset = analysis_grid[analysis_grid['spot_type'] == spot_type]
    color = colors.get(spot_type, '#cccccc')
    subset.plot(ax=ax, color=color, edgecolor='gray', linewidth=0.5, 
                alpha=0.8, label=f"{spot_type} ({len(subset)})")

ax.legend(loc='lower right', title='Clasificación')
ax.set_title('Análisis de Hot Spots - Densidad de Edificios\nIsla de Pascua', 
             fontsize=14, fontweight='bold')
ax.set_axis_off()

plt.tight_layout()
plt.savefig('../outputs/09_hotspots_gi_star.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Interpolación Espacial (IDW)

In [ ]:
# Interpolación IDW (Inverse Distance Weighting)
from scipy.interpolate import griddata

# Usar centroides de celdas con sus valores
analysis_grid['centroid_x'] = analysis_grid.geometry.centroid.x
analysis_grid['centroid_y'] = analysis_grid.geometry.centroid.y

# Puntos conocidos
points = analysis_grid[['centroid_x', 'centroid_y']].values
values = analysis_grid['building_count'].values

# Crear grilla para interpolación
xi = np.linspace(minx, maxx, 100)
yi = np.linspace(miny, maxy, 100)
Xi, Yi = np.meshgrid(xi, yi)

# Interpolación
Zi = griddata(points, values, (Xi, Yi), method='linear')

print("✅ Interpolación completada")

In [ ]:
# Mapa de superficie interpolada
fig, ax = plt.subplots(figsize=(14, 10))

# Contour plot
contour = ax.contourf(Xi, Yi, Zi, levels=20, cmap='YlOrRd', alpha=0.8)
plt.colorbar(contour, ax=ax, label='Densidad de Edificios')

# Añadir contornos
ax.contour(Xi, Yi, Zi, levels=10, colors='gray', linewidths=0.5, alpha=0.5)

# Límite
boundary_utm.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=3)

ax.set_title('Superficie de Densidad - Interpolación IDW\nIsla de Pascua', 
             fontsize=14, fontweight='bold')
ax.set_axis_off()

plt.tight_layout()
plt.savefig('../outputs/10_interpolacion_idw.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Análisis de Accesibilidad

In [ ]:
# Análisis de accesibilidad a amenidades
amenities_point = amenities_utm[amenities_utm.geometry.geom_type == 'Point'].copy()

if len(amenities_point) > 0:
    # Coordenadas de amenidades
    amenity_coords = np.array([[p.x, p.y] for p in amenities_point.geometry])
    
    # Para cada edificio, calcular distancia a amenidad más cercana
    building_coords = np.array([[p.x, p.y] for p in building_centroids.geometry])
    
    # Matriz de distancias
    dist_to_amenities = cdist(building_coords, amenity_coords)
    
    # Distancia mínima a cualquier amenidad
    min_dist = dist_to_amenities.min(axis=1)
    
    building_centroids['dist_to_amenity'] = min_dist
    
    print("📊 ACCESIBILIDAD A AMENIDADES")
    print("=" * 50)
    print(f"Distancia promedio: {min_dist.mean():.0f} m")
    print(f"Distancia mediana: {np.median(min_dist):.0f} m")
    print(f"Distancia máxima: {min_dist.max():.0f} m")
    print(f"\nEdificios a menos de 500m de amenidad: {(min_dist < 500).sum()} ({(min_dist < 500).mean()*100:.1f}%)")
    print(f"Edificios a menos de 1000m: {(min_dist < 1000).sum()} ({(min_dist < 1000).mean()*100:.1f}%)")
else:
    print("⚠️ No hay amenidades tipo Point")

In [ ]:
# Mapa de accesibilidad
if 'dist_to_amenity' in building_centroids.columns:
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Límite
    boundary_utm.plot(ax=ax, facecolor='lightgray', edgecolor='black', linewidth=2, alpha=0.3)
    
    # Edificios coloreados por distancia
    scatter = ax.scatter(
        building_coords[:,0], 
        building_coords[:,1],
        c=min_dist,
        cmap='RdYlGn_r',
        s=5,
        alpha=0.7
    )
    plt.colorbar(scatter, ax=ax, label='Distancia a amenidad más cercana (m)')
    
    # Amenidades
    ax.scatter(amenity_coords[:,0], amenity_coords[:,1], 
               c='blue', s=50, marker='*', label='Amenidades', zorder=5)
    
    ax.legend(loc='lower right')
    ax.set_title('Accesibilidad a Amenidades\nIsla de Pascua', fontsize=14, fontweight='bold')
    ax.set_axis_off()
    
    plt.tight_layout()
    plt.savefig('../outputs/11_accesibilidad.png', dpi=150, bbox_inches='tight')
    plt.show()

## 6. Kernel Density Estimation (KDE)

In [ ]:
from scipy.stats import gaussian_kde

# KDE para edificios
xy = np.vstack([building_coords[:,0], building_coords[:,1]])
kde = gaussian_kde(xy, bw_method=0.1)

# Evaluar en grilla
xx, yy = np.mgrid[minx:maxx:200j, miny:maxy:200j]
positions = np.vstack([xx.ravel(), yy.ravel()])
zz = np.reshape(kde(positions).T, xx.shape)

print("✅ KDE calculado")

In [ ]:
# Mapa de densidad KDE
fig, ax = plt.subplots(figsize=(14, 10))

# Contornos de densidad
cfset = ax.contourf(xx, yy, zz, levels=20, cmap='hot_r', alpha=0.8)
plt.colorbar(cfset, ax=ax, label='Densidad (KDE)')

# Límite
boundary_utm.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=3)

# Puntos
ax.scatter(building_coords[:,0], building_coords[:,1], c='white', s=1, alpha=0.3)

ax.set_title('Densidad de Edificios (Kernel Density Estimation)\nIsla de Pascua', 
             fontsize=14, fontweight='bold')
ax.set_axis_off()

plt.tight_layout()
plt.savefig('../outputs/12_kde_density.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Resumen del Análisis Geoestadístico

In [ ]:
print("\n" + "="*60)
print("📋 RESUMEN DEL ANÁLISIS GEOESTADÍSTICO")
print("="*60)

print("\n📍 ANÁLISIS DE PATRONES:")
print(f"   • Nearest Neighbor Index: {nn_index:.4f}")
print(f"   • Interpretación: {'Agrupado' if nn_index < 1 else 'Disperso' if nn_index > 1 else 'Aleatorio'}")

print("\n🔥 HOT SPOTS:")
if 'spot_type' in analysis_grid.columns:
    for st, count in analysis_grid['spot_type'].value_counts().items():
        print(f"   • {st}: {count} celdas")

print("\n📏 ACCESIBILIDAD:")
if 'dist_to_amenity' in building_centroids.columns:
    print(f"   • Distancia promedio a amenidad: {min_dist.mean():.0f} m")
    print(f"   • Edificios a < 500m: {(min_dist < 500).mean()*100:.1f}%")

print("\n📁 ARCHIVOS GENERADOS:")
print("   • outputs/08_nearest_neighbor.png")
print("   • outputs/09_hotspots_gi_star.png")
print("   • outputs/10_interpolacion_idw.png")
print("   • outputs/11_accesibilidad.png")
print("   • outputs/12_kde_density.png")

print("\n✅ Análisis geoestadístico completado!")